# Parse Nav.no sitemap

Denne notatboken analyserer
[nav.no/sitemap.xml](https://www.nav.no/sitemap.xml). Sitemap-en beskriver alle
åpne sider på [Nav.no](https://www.nav.no) og når de siste er oppdatert. Denne
informasjonen er fin å bruke når man skal laste ned innholdet.

In [ ]:
from rich import print

In [ ]:
# Filter for å fjerne stier som vi ikke er interessert i å indeksere
PATH_FILTER = (
    "/nav-og-samfunn/statistikk",  # Bob trenger ikke å kunne svare om statistikk
    "/nav-og-samfunn/kunnskap",  # Trenger ikke å svare om kunnskapsinnhold heller
)
# Filtrer ut sidetyper som ikke gir så mye mening at Bob bruker
PAGE_TYPE_FILTER = (
    "no.nav.navno:area-page",  # Oversikt over et tema som for det meste inneholder lenker: "/helse"
    "no.nav.navno:form-intermediate-step",  # Ser ut til å bare være knapp for å sende inn søknad: "/start/soknad-aap"
    "no.nav.navno:front-page",  # De store forsidene, disse har bare lenker videre: "www.nav.no/"
    "no.nav.navno:front-page-nested",  # Oversiktsside med bare lenker: "/utbetalinger"
    "no.nav.navno:generic-page",  # Lenke side: "/send-beskjed-om-syk"
    "no.nav.navno:large-table",  # Store tabeller med statistikk
    "no.nav.navno:main-article",  # Ser ut til å være artikler/nyheter: "https://www.nav.no/no/nav-og-samfunn/samarbeid/hjelpemidler/leverandorer-av-hjelpemidler/innovasjon-og-utvikling"
    "no.nav.navno:office-page",  # Kontaktinformasjon til lokalkontor: "/kontor/nav-sor-helgeland"
    "no.nav.navno:page-list",  # Ser ut som en lenke side: "https://www.nav.no/no/nav-og-samfunn/samarbeid/leger-og-andre-behandlere/om-sykmelding-for-leger"
    "no.nav.navno:press-landing-page",  # Kontaktinformasjon for presse (bare 1 side): "/samarbeidspartner/presse"
    "no.nav.navno:section-page",  # Lenkeside (bare 2 eksempler): "/se/samegiella"
)
# Bare ta med artikler på følgende språk
LANGUAGE_INCLUDE = (
    "no",
    "nn",
)

## Leser inn sitemap fra internett

In [ ]:
from navno_notebooks.sitemap import parse_sitemap

sitemap = await parse_sitemap("https://www.nav.no/sitemap.xml")

## Filtrerer ut sider

In [ ]:
from yarl import URL

to_index: list[URL] = []

for site in sitemap:
    if not any([path in site.loc.path for path in PATH_FILTER]):
        url = URL(str(site.loc))
        to_index.append(url)
print(
    f"Antall sider etter filtrering: {len(to_index)} "
    f"({len(to_index) / len(sitemap):.1%} av alle sider)"
)

## Hent ut JSON data

In [ ]:
from rich.progress import Progress

from navno_notebooks.metadata import Metadata, fetch_metadata

index: list[Metadata] = []
could_not_reach: list[URL] = []

with Progress() as prog:
    fetch_task = prog.add_task("Henter metadata", total=len(to_index))
    to_filter: list[Metadata]
    async for meta in fetch_metadata(to_index):
        could_not_reach = meta.could_not_reach
        to_filter = meta.metadata
        prog.update(
            fetch_task, completed=len(meta.metadata) + len(meta.could_not_reach)
        )
    filter_task = prog.add_task("Filtrerer sider", total=len(to_filter))
    for metadata in to_filter:
        if metadata.type in PAGE_TYPE_FILTER:
            continue
        if metadata.language not in LANGUAGE_INCLUDE:
            continue
        index.append(metadata)
        prog.advance(filter_task)

print(
    f"Endte med {len(index)} sider å indeksere etter filtrering på type ({len(index) / len(sitemap):.1%} av alle sider)"
)

In [ ]:
print(f"{len(could_not_reach)} sider kunne ikke nåes")

## Skriv til lokal JSON

In [ ]:
import json

from rich.prompt import Prompt

if local_file := Prompt.ask("Filnavn å skrive indeks til", default="index.json"):
    if not local_file.endswith(".json"):
        local_file += ".json"
    with open(local_file, mode="w") as fil:
        json.dump([site.model_dump(mode="json") for site in index], fil)